In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [3]:
model_id = "../models/Mixtral-8x7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [4]:
model = AutoModelForCausalLM.from_pretrained(model_id)

Loading checkpoint shards:  11%|█         | 2/19 [01:33<13:22, 47.22s/it]

: 